# Inline Functions

This is actually the name of the functions that we have encountered to date.  It is the term given to functions which are written directly into a SQL statement (such as in the SELECT and WHERE clauses).

Prime examples of there are:
* Year()
* CONCAT()
* ISNULL()

These are not ideal for re-usability and centralised code as they are generally system functions and out of our control.  For example, if we wished to see CONCAT used in a different way we cannot change the implementation that Microsoft has provided.  All we could do is to switch the function that we are using to another one, but that would involve a manual change to all code using the function.

However, they are very good for ad-hoc code, testing, and development.

Performance of these is very good although they can sometimes cause problems when used incorrectly as part of a WHERE clause (which we will briefly touch on below).

#### Examples

In the most basic form we could have the following fullName definition:

In [ ]:
select firstName, lastName, (firstName + ' ' + lastName) as fullName
from JupyterDatabase.hr.employee;

Likewise we commonly see the following style of query, using YEAR as our inline function:

In [ ]:
select year(orderDate) orderYear, sum(totalDue) as totalSales
from JupyterDatabase.sales.salesOrderHeader
order by orderYear;

#### Performance Notes

However, this is a good opportunity to touch on performance.

Although functions such as 'YEAR' are built into the SQL Server engine and therefore are highly performant when used in the above example, they do have a down side which can cause major performance problems.

SQL Server has a mechanism called indexing (which is discussed in depth as part of performance and not within basic T-SQL) which it uses to find data quickly within large tables.

Imagine a table with several million records spread across 50 years.  Using an Index SQL Server can easily locate and find data for a given year in the same way that we would use the index of a book to locate information within its pages.

However, now we'll consider the following query:

In [ ]:
select year(orderDate) orderYear, sum(totalDue) as totalSales
from JupyterDatabase.sales.salesOrderHeader
where year(orderDate) = 2019
order by orderYear;

The problem is that SQL Server will see this in the following way:

'I need to get all orderDate values and then obtain their year.  Once done, I can then find the records I need'.

This is incredibly ineffient as it means that SQL Server needs to read every record in the table and therefore cannot use any available indexes.

To remedy this we should remember the coding principle that we do not 'hide' columns from SQL Server.  The better way to write this would be as follows:

In [ ]:
select year(orderDate) orderYear, sum(totalDue) as totalSales
from JupyterDatabase.sales.salesOrderHeader
where orderDate between '2019-01-01' and '2019-12-31 23:59:59.999'
order by orderYear;

Using this method SQL Server knows we need a specific set of orderDate values and can utilise any available indexes to search for these rather than needing to read the table in entirety.